In [1]:
import ee
from cloud_mask import maskS2clouds, mask_cloudy_pixels
from config import startDate, endDate, region, cloud_cover, Map, clip_to_roi, max_cloud_probability
from Indices import NDVI_img, EVI_img, LSWI_img, SRWI_img, SRTI_img, NDTI_img, CRCI_img, MCRC_img, SAVI_img, NDSVI_img, NDSI_img,add_hillshade
from Date_format import format_date
from merge_S2_MODIS import merge_images
import pandas as pd
from meta_data_to_pandas import extract_metadata
from modis_projection import reproject_s2_to_modis_500m,reproject_s2_to_modis_scale_10m,reproject_s2_to_wgs84
ee.Authenticate()
ee.Initialize()


In [ ]:

# Load collections

s2_cloud_prob = ee.ImageCollection("COPERNICUS/S2_SR").filterDate(startDate, endDate).filterBounds(region).map(lambda img: img.clip(region)).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover)).map(NDVI_img).map(EVI_img).map(LSWI_img).map(SRTI_img).map(NDTI_img).map(CRCI_img).map(MCRC_img).map(SAVI_img).map(NDSVI_img).map(NDSI_img).map(add_hillshade).map(format_date)

clouds = ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY").filterDate(startDate, endDate).filterBounds(region)

# Attach cloud mask to each image by matching system:index
def attach_cloud_mask(img):
    cloud = clouds.filter(ee.Filter.equals('system:index', img.get('system:index'))).first()
    return img.set('cloud_mask', cloud)

s2_with_clouds = s2_cloud_prob.map(attach_cloud_mask)

# Apply cloud pixel masking
s2 = s2_with_clouds.map(lambda img: mask_cloudy_pixels(img, max_cloud_prob=max_cloud_probability))

vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}
Map.addLayer(s2.first(), vis_params, 'Sentinel-2 RGB')
Map

c:\Users\vtiwari\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\ee\deprecation.py:215: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To make sure your code keeps working, please update it.
This dataset has been superseded by COPERNICUS/S2_SR_HARMONIZED

Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


Map(center=[44.0125043483778, -120.4227606155169], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
# MODIS preprocessing
modis = (ee.ImageCollection('MODIS/061/MCD43A3')
         .select('Albedo_WSA_shortwave').filterBounds(region).map(lambda img: img.clip(region))
         .filterDate(startDate, endDate)
         .map(format_date))

Map.addLayer(modis,{},'modis')
Map


Map(bottom=95608.0, center=[44.0125043483778, -120.4227606155169], controls=(WidgetControl(options=['position'…

In [ ]:
# join MODIS and S2 on common dates

# Join by date
join = ee.Join.inner()
date_filter = ee.Filter.equals(leftField='date', rightField='date')
joined = join.apply(s2, modis, date_filter)

In [ ]:
# Merge MODIS and S2 Collection to single collection
mergedCollection = ee.ImageCollection(joined.map(merge_images))

# Debug print
#print(mergedCollection.getInfo())
Map.addLayer(mergedCollection.first(),{},"merged")
Map


Map(bottom=95608.0, center=[44.0125043483778, -120.4227606155169], controls=(WidgetControl(options=['position'…

# Export the information about no. of common images (MODIS and S2) as csv
Run this is you would like to export else leave it

In [ ]:
# # Convert to FeatureCollection
# metadata_fc = ee.FeatureCollection(mergedCollection.map(extract_metadata))

# # Get data to client side
# metadata_list = metadata_fc.getInfo()['features']

# # Convert to pandas DataFrame
# image_data = pd.DataFrame([f['properties'] for f in metadata_list])

# # Optional: convert timestamp to datetime
# image_data['system_time_start'] = pd.to_datetime(image_data['system_time_start'], unit='ms')

# # Save to CSV if needed
# image_data.to_csv("merged_image_metadata.csv", index=False)

# # Print preview
# print(image_data.head())

         date                                                 id  \
0  2021-09-16  20210916T185021_20210916T185515_T10TEM_2021_09_16   
1  2021-09-16  20210916T185021_20210916T185515_T10TEN_2021_09_16   
2  2021-09-16  20210916T185021_20210916T185515_T10TEP_2021_09_16   
3  2021-09-16  20210916T185021_20210916T185515_T10TFM_2021_09_16   
4  2021-09-16  20210916T185021_20210916T185515_T10TFN_2021_09_16   

        system_time_start  
0 2021-09-16 19:02:58.175  
1 2021-09-16 19:02:44.351  
2 2021-09-16 19:02:32.578  
3 2021-09-16 19:02:55.284  
4 2021-09-16 19:02:40.719  


# Batch export the images

In [ ]:
import ee
import time

# -------------------------------
# 1. Task Throttler (fast + safe)
# -------------------------------
def throttle(max_active=15, sleep_time=5):
    """Allow only N active GEE tasks to run in parallel safely."""
    while True:
        active_count = 0
        for t in ee.batch.Task.list():
            try:
                if t.active():
                    active_count += 1
            except ee.EEException:
                # Task no longer exists or status unknown; skip it
                continue

        print(f"Active tasks: {active_count} / {max_active}", end="\r")
        if active_count < max_active:
            break
        time.sleep(sleep_time)

# ---------------------------------------------------------
# 2. Prepare Image List (NO .getInfo on full collection)
# ---------------------------------------------------------
export_count = 5000  # change to mergedCollection.size().getInfo() for all
imageList = mergedCollection.toList(export_count)

# ---------------------------------------------------------
# 3. EXPORT LOOP (fast, parallel, prints status)
# ---------------------------------------------------------
for i in range(export_count):

    image = ee.Image(imageList.get(i))

    # ---------- Extract Safe Metadata ----------
    try:
        date_str = ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo()
        img_id = image.id().getInfo().replace("/", "_")
    except ee.EEException:
        print(f"⚠ Skipping image {i} due to missing metadata.")
        continue

    print(f"\n\n🔹 Exporting image {i+1}/{export_count} → {date_str} | {img_id}")

    # ---------- Prepare S2 Bands ----------
    s2Image = image.toFloat().select([
        'B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12',
        'ndvi','evi','lswi','srti','ndti','crci','mcrc',
        'savi','ndsvi','ndsi','hillshade'
    ])

    # ---------- MODIS Band ----------
    modisImage = image.toFloat().select('MODIS_Albedo_WSA_shortwave')

    # ---------- Reproject (server-side) ----------
    s2_500 = reproject_s2_to_wgs84(s2Image, 500)  # fixed scale
    s2_10 = reproject_s2_to_modis_scale_10m(s2Image, modisImage)
    s2_modis_merged = reproject_s2_to_modis_500m(s2Image, modisImage)

    # ---------- Create Task ----------
    task = ee.batch.Export.image.toDrive(
        image=s2_500,
        description=f"S2_500m_{date_str}_{img_id}",
        folder="MergedCollectionExports2021_testt",
        fileNamePrefix=f"S2_500m_{date_str}_{img_id}",
        maxPixels=1e13,
        fileFormat="GeoTIFF"
    )

    # ---------- Throttle BEFORE starting next task ----------
    throttle(max_active=15)

    # ---------- Start Task ----------
    try:
        task.start()
        print(f"✔ Task started: S2_500m_{date_str}_{img_id}")
    except ee.EEException as e:
        print(f"⚠ Failed to start task for {img_id}: {e}")
        continue




🔹 Exporting image 1/5000 → 2021-09-16 | 20210916T185021_20210916T185515_T10TEM_2021_09_16
✔ Task started: S2_500m_2021-09-16_20210916T185021_20210916T185515_T10TEM_2021_09_16


🔹 Exporting image 2/5000 → 2021-09-16 | 20210916T185021_20210916T185515_T10TEN_2021_09_16
